<a href="https://colab.research.google.com/github/xu842251462/verilog-llm/blob/main/VGen_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [1]:

!pip install git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-48maorgl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-48maorgl
  Resolved https://github.com/huggingface/transformers.git to commit 3457e8e73e4f5532cc69059682b1ba4484d7e7e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.54.0.dev0-py3-none-any.whl size=11788847 sha256=9f42284685e2914431908b863f7c01f0a526bdae607465d903020d907669dfb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-0xs_0xs3/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 30 21:09:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# Load model and tokenizer

In [4]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")


model_name = "shailja/CodeGen_2B_Verilog"

tokenizer = AutoTokenizer.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog")

model = AutoModelForCausalLM.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog").to(device)



Using GPU


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at shailja/fine-tuned-codegen-2B-Verilog were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.20.attn.causal_mask', 'transformer.h.21.attn.causal_mask', 'transformer.h.22.attn.causal_mask', 'transformer.h.23.attn.causal_mask', 'transformer.h.24.attn.causal_mask', 'transformer.h.25.attn.causal_mask', 'transformer.h.26.attn.causal_mask', 'transformer.h.27.attn.causal_mask', 'transformer.h.28.attn.causal_mask', 'transformer.h.29.attn.causal_mask', 'transformer.h.3.attn

# Try out the model for sampling Verilog

In [6]:

prompt = "//Write a module that implements a 2-to-1 Mux "
# prompt="// Design a 2-to-1 multiplexer.\n module mux( \n    input [4:0] a, b,\n    input sel,\n    output [4:0] out );\n // When sel=0, assign a to out. \n// When sel=1, assign b to out."
n_steps = 1
n = 5
end_pattern='endmodule'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# Sample
sample = model.generate(input_ids, max_length=128, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0], truncate_before_pattern=[r"endmodule"]) + end_pattern)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


//module half adder 
//input a,b,c
//output s,cout
//half adder module
module half_adder(a,b,s,cout);

input a,b;
output s,cout;

assign s = a ^ b;
assign cout = a & b;

endmodule


# Try out the model for sampling Verilog per token basis

In [7]:

iteration = dict()
with torch.no_grad():
    while True:

        iteration["Input"] = input_ids

        output = model(input_ids)

        # select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # print(sorted_ids)


        # store tokens with highest probability
        for choice_idx in range(n):
            token_id = sorted_ids[choice_idx]

            token_prob = next_token_probs[token_id].cpu().numpy()
            # print(token_id, token_prob)
            # print(tokenizer.decode(token_id.view(1,-1)))

            token_choice = (
                f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)"
            )

            iteration[f"{choice_idx+1}"] = token_choice

        #Append predictions to list
        input_ids = torch.cat([input_ids, sorted_ids[None,0, None]], dim=-1)

        # condition checks on stop_words detected
        if 'endmodule' in tokenizer.batch_decode(input_ids)[0]: break

print(tokenizer.batch_decode(input_ids)[0])

//module half adder 
//input a,b,c
//output s,cout
//half adder module
module half_adder(a,b,s,cout);

input a,b;
output s,cout;

assign s = a ^ b;
assign cout = a & b;

endmodule
